In [1]:
import pandas as pd

In [3]:
!pip install openpyxl


   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   --------- ----------------------------- 61.4/250.9 kB 825.8 kB/s eta 0:00:01
   ---------------------------------------  245.8/250.9 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 2.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
df = pd.read_excel("Consumer.xlsx")
df.head()


,order_id,order_date,customer_id,total_price,first_name,last_name,address_x,email_x,phone_number_x,order_item_id,...,amount,transaction_status,shipment_id,shipment_date,carrier,tracking_number,delivery_date,shipment_status,rating,order_value
0,11331,2023-11-17,1568,2180.01,James,Evelyn,"343 Lake Ave, Harbor City, VA",jevelyn1568@customer.com,555-349-7397,4044,...,919.00,Completed,11331,2023-11-21,FedEx,TRK138159,2023-11-25,Shipped,5.0,418.86
1,7181,2024-06-14,7386,1583.76,Mary,Christina,"939 Sand Ln, Gulf Shore, NE",mchristina7386@customer.com,555-487-2713,4053,...,597.92,Completed,7181,2024-06-19,DHL,TRK776300,2024-06-27,Pending,NaN,2151.48
2,14522,2024-03-10,6277,208.32,Mary,Lawrence,"242 Garden St, Liberty City, AL",mlawrence6277@customer.com,555-617-9529,4464,...,420.56,Failed,14522,2024-03-11,DHL,TRK527168,2024-03-18,Delivered,NaN,2599.68
3,14626,2024-06-08,7164,3036.12,Mary,Julia,"696 Anchor St, Port Town, MO",mjulia7164@customer.com,555-191-2041,4707,...,425.74,Failed,14626,2024-06-13,UPS,TRK514203,2024-06-18,Shipped,NaN,2503.20
4,14441,2024-07-29,1090,445.54,James,Nicole,"444 Magnolia Blvd, Mountain View, AZ",jnicole1090@customer.com,555-074-7845,343,...,619.12,Failed,14441,2024-08-03,DHL,TRK466889,2024-08-10,Pending,NaN,83.35


In [3]:
df.groupby('order_id')['product_name'].nunique().describe()


count    148.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: product_name, dtype: float64

In [4]:
transactions = (
    df.groupby('order_id')['category']
    .apply(lambda x: list(set(x)))
)
transactions.head()


order_id
155       [Electronics]
163         [Furniture]
321    [Home & Kitchen]
354       [Accessories]
389         [Furniture]
Name: category, dtype: object

In [6]:
transactions = df.groupby('order_id')['category'].apply(list)
transactions.head()


order_id
155       [Electronics]
163         [Furniture]
321    [Home & Kitchen]
354       [Accessories]
389         [Furniture]
Name: category, dtype: object

In [7]:
basket = pd.DataFrame(0, index=transactions.index, columns=pd.unique(df['category']))

for order_id, cats in transactions.items():
    for cat in set(cats):
        basket.loc[order_id, cat] = 1

basket.head()


,Home & Kitchen,Accessories,Furniture,Electronics
order_id,,,,
155,0,0,0,1
163,0,0,1,0
321,1,0,0,0
354,0,1,0,0
389,0,0,1,0


In [8]:
support = basket.sum() / len(basket)
support.sort_values(ascending=False)


Electronics       0.391892
Home & Kitchen    0.256757
Accessories       0.256757
Furniture         0.094595
dtype: float64

In [9]:
from itertools import combinations
from collections import Counter

pair_counts = Counter()

for _, row in basket.iterrows():
    items = basket.columns[row == 1]
    for pair in combinations(items, 2):
        pair_counts[pair] += 1

pair_df = pd.DataFrame(pair_counts.items(), columns=['Category Pair', 'Frequency'])
pair_df.sort_values(by='Frequency', ascending=False)


,Category Pair,Frequency


In [10]:
pair_df['Support'] = pair_df['Frequency'] / len(basket)
pair_df


,Category Pair,Frequency,Support
